# NeuralQA
- NeuralQA: A Usable Library for Question Answering on Large Datasets with BERT 
- [documentation](https://victordibia.com/neuralqa/)
- paper
- [github](https://github.com/victordibia/neuralqa)
- [Demo](https://neuralqa.fastforwardlabs.com/#/)



steps to setup?
- config.yaml file



```
reader:
  title: Reader
  selected: twmkn9/distilbert-base-uncased-squad2
  options:
    - name: DistilBERT SQUAD2
      value: twmkn9/distilbert-base-uncased-squad2
      type: distilbert
    - name: BERT SQUAD2
      value: deepset/bert-base-cased-squad2
      type: bert
  ```

Run: `python research_papers/neuralQA.py`

```
------------------------------ GET ANSWER
{'answers': [{'answer': 'to protect people ’ s right to privacy and freedom from unreasonable intrusions by the government', 'took': 0.24120783805847168, 'start_probability': '0.8377399', 'end_probability': '0.0031696134', 'probability': '0.4220395558513701', 'question': 'what is the goal of the fourth amendment?', 'context': "the fourth amendment of the u. s. constitution provides that the right of the people to be secure in their persons, houses, papers, and effects, against unreasonable searches and seizures, shall not be violated, and no warrants shall issue, but upon probable cause, supported by oath or affirmation, and particularly describing the place to be searched, and the persons or things to be seized.'the ultimate goal of this provision is  <em>to protect people ’ s right to privacy and freedom from unreasonable intrusions by the government</em> . however, the fourth amendment does not guarantee protection from all searches and seizures, but only those done by the government and deemed unreasonable under the law", 'index': 0}], 'took': 0.24695920944213867}
------------------------------ GET EXPLANATION
{'gradients': [{'gradient': 0.7776250839233398, 'token': 'what', 'token_type': 0}, {'gradient': 0.38133350014686584, 'token': 'is', 'token_type': 0}, {'gradient': 0.4833914041519165, 'token': 'the', 'token_type': 0}, {'gradient': 1.0, 'token': 'goal', 'token_type': 0}, {'gradient': 0.4539875388145447, 'token': 'of', 'token_type': 0}, {'gradient': 0.22548739612102509, 'token': 'the', 'token_type': 0}, {'gradient': 0.5164982676506042, 'token': 'fourth', 'token_type': 0}, {'gradient': 0.5774044394493103, 'token': 'amendment', 'token_type': 0}, {'gradient': 0.4014104902744293, 'token': '?', 'token_type': 0}, {'gradient': 0.08653238415718079, 'token': 'the', 'token_type': 1}, {'gradient': 0.4406396746635437, 'token': 'fourth', 'token_type': 1}, {'gradient': 0.4501517415046692, 'token': 'amendment', 'token_type': 1}, {'gradient': 0.09730646014213562, 'token': 'of', 'token_type': 1}, {'gradient': 0.05219585448503494, 'token': 'the', 'token_type': 1}, {'gradient': 0.1063244566321373, 'token': 'u', 'token_type': 1}, {'gradient': 0.08209721744060516, 'token': '.', 'token_type': 1}, {'gradient': 0.11832371354103088, 'token': 's', 'token_type': 1}, {'gradient': 0.12593641877174377, 'token': '.', 'token_type': 1}, {'gradient': 0.182209774851799, 'token': 'constitution', 'token_type': 1}, {'gradient': 0.22337506711483002, 'token': 'provides', 'token_type': 1}, {'gradient': 0.09926066547632217, 'token': 'that', 'token_type': 1}, {'gradient': 0.04957519471645355, 'token': 'the', 'token_type': 1}, {'gradient': 0.06091606616973877, 'token': 'right', 'token_type': 1}, {'gradient': 0.04879080131649971, 'token': 'of', 'token_type': 1}, {'gradient': 0.039429035037755966, 'token': 'the', 'token_type': 1}, {'gradient': 0.07061353325843811, 'token': 'people', 'token_type': 1}, {'gradient': 0.035968974232673645, 'token': 'to', 'token_type': 1}, {'gradient': 0.05321972444653511, 'token': 'be', 'token_type': 1}, {'gradient': 0.08256882429122925, 'token': 'secure', 'token_type': 1}, {'gradient': 0.06096405163407326, 'token': 'in', 'token_type': 1}, {'gradient': 0.037246234714984894, 'token': 'their', 'token_type': 1}, {'gradient': 0.0629529058933258, 'token': 'persons', 'token_type': 1}, {'gradient': 0.04519990086555481, 'token': ',', 'token_type': 1}, {'gradient': 0.06783896684646606, 'token': 'houses', 'token_type': 1}, {'gradient': 0.040804434567689896, 'token': ',', 'token_type': 1}, {'gradient': 0.06289409846067429, 'token': 'papers', 'token_type': 1}, {'gradient': 0.03958077356219292, 'token': ',', 'token_type': 1}, {'gradient': 0.051120467483997345, 'token': 'and', 'token_type': 1}, {'gradient': 0.07284311205148697, 'token': 'effects', 'token_type': 1}, {'gradient': 0.0512305311858654, 'token': ',', 'token_type': 1}, {'gradient': 0.0562400259077549, 'token': 'against', 'token_type': 1}, {'gradient': 0.18519803881645203, 'token': 'unreasonable', 'token_type': 1}, {'gradient': 0.06078466400504112, 'token': 'searches', 'token_type': 1}, {'gradient': 0.07405954599380493, 'token': 'and', 'token_type': 1}, {'gradient': 0.07777302712202072, 'token': 'seizures', 'token_type': 1}, {'gradient': 0.06557746231555939, 'token': ',', 'token_type': 1}, {'gradient': 0.09869323670864105, 'token': 'shall', 'token_type': 1}, {'gradient': 0.07527294754981995, 'token': 'not', 'token_type': 1}, {'gradient': 0.04569385573267937, 'token': 'be', 'token_type': 1}, {'gradient': 0.10462981462478638, 'token': 'violated', 'token_type': 1}, {'gradient': 0.0642581656575203, 'token': ',', 'token_type': 1}, {'gradient': 0.05537228658795357, 'token': 'and', 'token_type': 1}, {'gradient': 0.06339304149150848, 'token': 'no', 'token_type': 1}, {'gradient': 0.045494332909584045, 'token': 'warrants', 'token_type': 1}, {'gradient': 0.07791747152805328, 'token': 'shall', 'token_type': 1}, {'gradient': 0.04790063947439194, 'token': 'issue', 'token_type': 1}, {'gradient': 0.05423518642783165, 'token': ',', 'token_type': 1}, {'gradient': 0.054544705897569656, 'token': 'but', 'token_type': 1}, {'gradient': 0.04602178931236267, 'token': 'upon', 'token_type': 1}, {'gradient': 0.07888590544462204, 'token': 'probable', 'token_type': 1}, {'gradient': 0.07856247574090958, 'token': 'cause', 'token_type': 1}, {'gradient': 0.11361145973205566, 'token': ',', 'token_type': 1}, {'gradient': 0.06214277073740959, 'token': 'supported', 'token_type': 1}, {'gradient': 0.05307868868112564, 'token': 'by', 'token_type': 1}, {'gradient': 0.09184949845075607, 'token': 'oath', 'token_type': 1}, {'gradient': 0.060711536556482315, 'token': 'or', 'token_type': 1}, {'gradient': 0.04753934592008591, 'token': 'affirmation', 'token_type': 1}, {'gradient': 0.07517804205417633, 'token': ',', 'token_type': 1}, {'gradient': 0.06382709741592407, 'token': 'and', 'token_type': 1}, {'gradient': 0.10078532993793488, 'token': 'particularly', 'token_type': 1}, {'gradient': 0.07376571744680405, 'token': 'describing', 'token_type': 1}, {'gradient': 0.03218425065279007, 'token': 'the', 'token_type': 1}, {'gradient': 0.047836221754550934, 'token': 'place', 'token_type': 1}, {'gradient': 0.051581498235464096, 'token': 'to', 'token_type': 1}, {'gradient': 0.03697744384407997, 'token': 'be', 'token_type': 1}, {'gradient': 0.0848195031285286, 'token': 'searched', 'token_type': 1}, {'gradient': 0.0867309719324112, 'token': ',', 'token_type': 1}, {'gradient': 0.060603126883506775, 'token': 'and', 'token_type': 1}, {'gradient': 0.0426025353372097, 'token': 'the', 'token_type': 1}, {'gradient': 0.05684027075767517, 'token': 'persons', 'token_type': 1}, {'gradient': 0.06187772750854492, 'token': 'or', 'token_type': 1}, {'gradient': 0.04903043434023857, 'token': 'things', 'token_type': 1}, {'gradient': 0.04995299503207207, 'token': 'to', 'token_type': 1}, {'gradient': 0.06450381129980087, 'token': 'be', 'token_type': 1}, {'gradient': 0.13170987367630005, 'token': 'seized', 'token_type': 1}, {'gradient': 0.27072256803512573, 'token': '.', 'token_type': 1}, {'gradient': 0.3593484163284302, 'token': "'", 'token_type': 1}, {'gradient': 0.2004624456167221, 'token': 'the', 'token_type': 1}, {'gradient': 0.5591768026351929, 'token': 'ultimate', 'token_type': 1}, {'gradient': 0.6059504151344299, 'token': 'goal', 'token_type': 1}, {'gradient': 0.22130262851715088, 'token': 'of', 'token_type': 1}, {'gradient': 0.2113972008228302, 'token': 'this', 'token_type': 1}, {'gradient': 0.2291385978460312, 'token': 'provision', 'token_type': 1}, {'gradient': 0.3722270131111145, 'token': 'is', 'token_type': 1}, {'gradient': 0.3713296949863434, 'token': 'to', 'token_type': 1}, {'gradient': 0.37173280119895935, 'token': 'protect', 'token_type': 1}, {'gradient': 0.16041772067546844, 'token': 'people', 'token_type': 1}, {'gradient': 0.15185077488422394, 'token': '’', 'token_type': 1}, {'gradient': 0.08072197437286377, 'token': 's', 'token_type': 1}, {'gradient': 0.1538591831922531, 'token': 'right', 'token_type': 1}, {'gradient': 0.050815191119909286, 'token': 'to', 'token_type': 1}, {'gradient': 0.167207270860672, 'token': 'privacy', 'token_type': 1}, {'gradient': 0.14042355120182037, 'token': 'and', 'token_type': 1}, {'gradient': 0.15495669841766357, 'token': 'freedom', 'token_type': 1}, {'gradient': 0.16938287019729614, 'token': 'from', 'token_type': 1}, {'gradient': 0.1100737527012825, 'token': 'unreasonable', 'token_type': 1}, {'gradient': 0.11692174151539803, 'token': 'intrusions', 'token_type': 1}, {'gradient': 0.10463490337133408, 'token': 'by', 'token_type': 1}, {'gradient': 0.1022951602935791, 'token': 'the', 'token_type': 1}, {'gradient': 0.4751105010509491, 'token': 'government', 'token_type': 1}, {'gradient': 0.48005521297454834, 'token': '.', 'token_type': 1}, {'gradient': 0.3730919063091278, 'token': 'however', 'token_type': 1}, {'gradient': 0.2522304356098175, 'token': ',', 'token_type': 1}, {'gradient': 0.18487226963043213, 'token': 'the', 'token_type': 1}, {'gradient': 0.3077232837677002, 'token': 'fourth', 'token_type': 1}, {'gradient': 0.24925129115581512, 'token': 'amendment', 'token_type': 1}, {'gradient': 0.1148197203874588, 'token': 'does', 'token_type': 1}, {'gradient': 0.12900976836681366, 'token': 'not', 'token_type': 1}, {'gradient': 0.16723978519439697, 'token': 'guarantee', 'token_type': 1}, {'gradient': 0.0805215984582901, 'token': 'protection', 'token_type': 1}, {'gradient': 0.05996048450469971, 'token': 'from', 'token_type': 1}, {'gradient': 0.046615201979875565, 'token': 'all', 'token_type': 1}, {'gradient': 0.07966292649507523, 'token': 'searches', 'token_type': 1}, {'gradient': 0.09778250008821487, 'token': 'and', 'token_type': 1}, {'gradient': 0.13690157234668732, 'token': 'seizures', 'token_type': 1}, {'gradient': 0.10977756977081299, 'token': ',', 'token_type': 1}, {'gradient': 0.14516036212444305, 'token': 'but', 'token_type': 1}, {'gradient': 0.07196351140737534, 'token': 'only', 'token_type': 1}, {'gradient': 0.09799711406230927, 'token': 'those', 'token_type': 1}, {'gradient': 0.055412378162145615, 'token': 'done', 'token_type': 1}, {'gradient': 0.05443686246871948, 'token': 'by', 'token_type': 1}, {'gradient': 0.06119629368185997, 'token': 'the', 'token_type': 1}, {'gradient': 0.11526168137788773, 'token': 'government', 'token_type': 1}, {'gradient': 0.07573366165161133, 'token': 'and', 'token_type': 1}, {'gradient': 0.09001242369413376, 'token': 'deemed', 'token_type': 1}, {'gradient': 0.0890301764011383, 'token': 'unreasonable', 'token_type': 1}, {'gradient': 0.07160928845405579, 'token': 'under', 'token_type': 1}, {'gradient': 0.056089501827955246, 'token': 'the', 'token_type': 1}, {'gradient': 0.056684162467718124, 'token': 'law', 'token_type': 1}, {'gradient': 0.13379959762096405, 'token': '.', 'token_type': 1}], 'answer': 'to protect people ’ s right to privacy and freedom from unreasonable intrusions by the government', 'question': 'what is the goal of the fourth amendment?'}
```